In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

# CGV

## Event List

In [2]:
def get_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        movie_name = m.group()[1:-1]
        goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [3]:
event_df = get_EventList(goods_filter="TTT")
event_df

,event_idx,goods_idx,movie,goods,period
0,202403253872,2024032538723859,고질라 X 콩-뉴 엠파이어,TTT,2024.03.27 ~ 2024.04.26
5,202403183820,2024031838203819,극장판 스파이 패밀리 코드-화이트,TTT,2024.03.20 ~ 2024.04.19
6,202404013900,2024040139003887,기동전사 건담 시드 프리덤,TTT,2024.04.03 ~ 2024.05.02
13,202404093946,2024040939463943,슈가│어거스트 디 투어~,TTT 증정,2024.04.10 ~ 2024.04.16
23,202404013901,2024040139013888,유미의 세포들 더 무비,TTT,2024.04.03 ~ 2024.05.02
26,202404083925,2024040839253922,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
30,202403043767,2024030437673750,패스트 라이브즈,TTT,2024.03.06 ~ 2024.04.21


## Goods

- type2: 재고보유가능성높음
- type3: 재고소진임박
- type4: 마감

In [4]:
def get_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [5]:
giveawayIndex = '2024032538723859'
Theaters = get_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강남,0056,type2
1,CGV 강변,0001,type4
2,CGV 구로,0010,type2
3,CGV 동대문,0252,type2
4,CGV 등촌,0230,type2
...,...,...,...
59,CGV 서전주,0225,type2
60,CGV 전주고사,0213,type4
61,CGV 목포평화광장,0280,type2
62,CGV 순천신대,0268,type4


In [8]:
def Get_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_EventList(movie_filter=movie_filter, goods_filter=goods_filter)
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        giveawayIndex = event_df.loc[idx]["goods_idx"]
        Theaters = get_Theaters(giveawayIndex)
        Theaters["movie"] =  event_df.loc[idx]["movie"]
        Theaters["goods"] =  event_df.loc[idx]["goods"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [12]:
GoodsStatus = Get_GoodsStatus(movie_filter="쿵푸팬더", goods_filter="TTT")

In [13]:
GoodsStatus

,TheaterName,TheaterCode,CountTypeCode,movie,goods,period
0,CGV 강남,0056,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
1,CGV 강변,0001,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
2,CGV 구로,0010,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
3,CGV 동대문,0252,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
4,CGV 등촌,0230,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
...,...,...,...,...,...,...
61,CGV 서전주,0225,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
62,CGV 전주고사,0213,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
63,CGV 목포평화광장,0280,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
64,CGV 순천신대,0268,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09


In [11]:
GoodsStatus["movie"].unique()

array(['고질라 X 콩-뉴 엠파이어', '극장판 스파이 패밀리 코드-화이트', '기동전사 건담 시드 프리덤',
       '슈가│어거스트 디 투어~', '유미의 세포들 더 무비', '쿵푸팬더4', '패스트 라이브즈'], dtype=object)

# LotteCinema

In [26]:
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetEventLists","channelType":"","osType":"","osVersion":"","EventClassificationCode":"20","SearchText":"","CinemaID":"","PageNo":1,"PageSize":100,"MemberNo":"0"}'
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)

In [30]:
pd.DataFrame(loads["Items"])

,EventID,EventName,EventClassificationCode,EventTypeCode,EventTypeName,ProgressStartDate,ProgressEndDate,ImageUrl,ImageAlt,ImageDivisionCode,...,CinemaName,CinemaAreaCode,CinemaAreaName,DevTemplateYN,CloseNearYN,RemainsDayCount,EventWinnerYN,EventSeq,EventCntnt,EventNtc
0,201010016924190,<꽃다발 같은 사랑을 했다>2주차 현장증정이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/8905b...,<꽃다발 같은 사랑을 했다>2주차 현장증정이벤트,20,...,,,,0,0,8,0,29,,
1,201010016924191,<악은 존재하지 않는다>4주차 현장증정이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/41683...,<악은 존재하지 않는다>4주차 현장증정이벤트,20,...,,,,0,0,8,0,28,,
2,201010016924192,<땅에 쓰는 시>1주차 현장 증정 이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/d3c71...,<땅에 쓰는 시>1주차 현장 증정 이벤트,20,...,,,,0,0,8,0,27,,
3,201010016924193,<기생수 파트2>1주차 현장증정 이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/945eb...,<기생수 파트2>1주차 현장증정 이벤트,20,...,,,,0,0,8,0,26,,
4,201010016924194,<유미의 세포들>3주차 현장증정이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/16d74...,<유미의 세포들>3주차 현장증정이벤트,20,...,,,,0,0,8,0,25,,
5,201010016924195,<남은 인생10년> 3주차 현장증정이벤트,20,101,정보전달형(공지),2024.04.17,2024.04.23,http://cf.lottecinema.co.kr//Media/Event/26f5a...,<남은 인생10년> 3주차 현장증정이벤트,20,...,,,,0,0,8,0,24,,
6,201010016924183,<쿵푸팬더4>1주차 주말 현장 증정이벤트,20,101,정보전달형(공지),2024.04.13,2024.04.19,http://cf.lottecinema.co.kr//Media/Event/5ad46...,<쿵푸팬더4>1주차 주말 현장 증정이벤트,20,...,,,,0,0,4,0,23,,
7,201010016924179,<기동전사 건담>2주차 주말증정이벤트,20,101,정보전달형(공지),2024.04.13,2024.04.19,http://cf.lottecinema.co.kr//Media/Event/efd8d...,<기동전사 건담>2주차 주말증정이벤트,20,...,,,,0,0,4,0,22,,<FONT color=#ff0000>\r\n<P><STRONG><FONT color...
8,201010016924186,<스파이패밀리>4주차 주말증정이벤트,20,101,정보전달형(공지),2024.04.13,2024.04.19,http://cf.lottecinema.co.kr//Media/Event/96499...,<스파이패밀리>4주차 주말증정이벤트,20,...,,,,0,0,4,0,21,,<FONT color=#ff0000>\r\n<P><STRONG><FONT color...
9,201010016924187,<남은 인생 10년>2주차 주말증정이벤트,20,101,정보전달형(공지),2024.04.13,2024.04.19,http://cf.lottecinema.co.kr//Media/Event/c7909...,<남은 인생 10년>2주차 주말증정이벤트,20,...,,,,0,0,4,0,20,,<P><STRONG><FONT color=#ff0000>- 상영 편성 상황에 따라 ...


In [56]:
# EventID = "201010016924170"
EventID = "201010016924184"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

[]

In [53]:
EventID = "201010016924176"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

[{'EventID': '201010016924176',
  'FrGiftID': '9109',
  'FrGiftNm': '<남은 인생 10년>2주차 굿즈(4/11목~4/16화,인생네컷)'}]

In [57]:
EventID = "201010016924170"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

[{'EventID': '201010016924170',
  'FrGiftID': '9081',
  'FrGiftNm': '<쿵푸팬더4>시그니처 아트카드(4/10수~4/16화, 회차당 수령제한O)'}]

In [62]:
EventID = "201010016924176"
FrGiftID = "9109"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
loads["CinemaDivisionGoods"]

[{'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1004',
  'CinemaNameKR': '건대입구',
  'CinemaNameUS': 'Konkuk Univ',
  'SortSequence': 6,
  'Cnt': 100,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1009',
  'CinemaNameKR': '김포공항',
  'CinemaNameUS': "Gimpo Int'l Airport",
  'SortSequence': 7,
  'Cnt': 100,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1003',
  'CinemaNameKR': '노원',
  'CinemaNameUS': 'Nowon',
  'SortSequence': 8,
  'Cnt': 100,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1007',
  'CinemaNameKR': '신림',
  'CinemaNameUS': 'Sillim',
  'SortSequence': 18,
  'Cnt': 50,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'Cin

In [61]:
EventID = "201010016924170"
FrGiftID = "9081"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
loads["CinemaDivisionGoods"]

[{'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1013',
  'CinemaNameKR': '가산디지털',
  'CinemaNameUS': 'Gasan Digital',
  'SortSequence': 1,
  'Cnt': 40,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1004',
  'CinemaNameKR': '건대입구',
  'CinemaNameUS': 'Konkuk Univ',
  'SortSequence': 6,
  'Cnt': 0,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1009',
  'CinemaNameKR': '김포공항',
  'CinemaNameUS': "Gimpo Int'l Airport",
  'SortSequence': 7,
  'Cnt': 0,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  'CinemaID': '1003',
  'CinemaNameKR': '노원',
  'CinemaNameUS': 'Nowon',
  'SortSequence': 8,
  'Cnt': 0,
  'DetailDivisionNameKR': '서울',
  'DetailDivisionNameUS': 'Seoul'},
 {'DivisionCode': 1,
  'DetailDivisionCode': '0001',
  '